In [77]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import datetime

# Importing corrected Dataset

In [78]:
df=pd.read_csv("C:\\Users\\erohcho\\OneDrive - Ericsson AB\\Project NSE\\NSE PROTOTYPE\\MasterFile_Updated.csv")
df.head(2)

,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,SECTOR
0,20MICRONS,58.95,60.70,57.90,58.75,156506,9316661.05,1/1/2018 0:00,1527,Mining
1,20MICRONS,58.75,60.55,57.65,58.05,173946,10190754.15,1/2/2018 0:00,994,Mining


In [79]:
df['TIMESTAMP'] = df['TIMESTAMP'].apply(pd.to_datetime)
df.set_index('TIMESTAMP',inplace=True)
df.head(2)

,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,SECTOR
TIMESTAMP,,,,,,,,,
2018-01-01,20MICRONS,58.95,60.70,57.90,58.75,156506,9316661.05,1527,Mining
2018-01-02,20MICRONS,58.75,60.55,57.65,58.05,173946,10190754.15,994,Mining


In [55]:
ewma = pd.Series.ewm

# Grouping by SYMBOL [SMA_20 & Bollinger Band]

In [80]:
df['SMA_20'] = df.groupby('SYMBOL')['CLOSE'].transform(lambda x: x.rolling(20,1).mean())
df['Std_Dev'] = df.groupby('SYMBOL')['CLOSE'].transform(lambda x: x.rolling(20,1).std())

In [81]:
df['BB_High']=df['SMA_20']+2*df['Std_Dev'].replace(np.nan, 0)
df['BB_Low']=df['SMA_20']-2*df['Std_Dev'].replace(np.nan, 0)
df.head(2)

,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,SECTOR,SMA_20,Std_Dev,BB_High,BB_Low
TIMESTAMP,,,,,,,,,,,,,
2018-01-01,20MICRONS,58.95,60.70,57.90,58.75,156506,9316661.05,1527,Mining,58.75,NaN,58.750000,58.750000
2018-01-02,20MICRONS,58.75,60.55,57.65,58.05,173946,10190754.15,994,Mining,58.40,0.494975,59.389949,57.410051


In [25]:
#df.reset_index()

In [82]:
ewma = pd.Series.ewm

In [83]:
def rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [87]:
df['EMA_12'] = ewma(df["CLOSE"], span=12).mean()
df['EMA_26'] = ewma(df["CLOSE"], span=26).mean()
df['EMA_9'] = ewma(df['CLOSE'],span=9).mean()
df['MACD']=df['EMA_12']-df['EMA_26']

In [88]:
df.head()

,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,SECTOR,SMA_20,Std_Dev,BB_High,BB_Low,EMA_12,EMA_26,EMA_9,MACD
TIMESTAMP,,,,,,,,,,,,,,,,,
2018-01-01,20MICRONS,58.95,60.70,57.90,58.75,156506,9316661.05,1527,Mining,58.750000,NaN,58.750000,58.750000,58.750000,58.750000,58.750000,0.000000
2018-01-02,20MICRONS,58.75,60.55,57.65,58.05,173946,10190754.15,994,Mining,58.400000,0.494975,59.389949,57.410051,58.370833,58.386538,58.361111,-0.015705
2018-01-03,20MICRONS,58.70,60.50,58.50,59.20,191320,11345473.80,1004,Mining,58.666667,0.579511,59.825689,57.507644,58.694457,58.678807,58.704918,0.015650
2018-01-04,20MICRONS,59.45,63.20,58.60,62.60,812025,50057113.35,5189,Mining,59.650000,2.022787,63.695574,55.604426,59.927284,59.775001,60.024390,0.152283
2018-01-05,20MICRONS,64.80,67.00,63.00,63.60,1129498,73646779.85,7645,Mining,60.440000,2.487820,65.415641,55.464359,60.925148,60.662034,61.088053,0.263114


In [90]:
df['RSI_20'] = rsiFunc(df['CLOSE'].values, 20)

In [94]:
df.tail()

,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,SECTOR,SMA_20,Std_Dev,BB_High,BB_Low,EMA_12,EMA_26,EMA_9,MACD,RSI_20
TIMESTAMP,,,,,,,,,,,,,,,,,,
2018-02-06,ZYLOG,3.55,3.9,3.55,3.7,113075,422604.95,242,Computers,4.1400,0.235417,4.610834,3.669166,21.396360,179.666640,8.010826,-158.270280,12.661952
2018-02-07,ZYLOG,3.90,3.9,3.75,3.8,104979,401737.05,168,Computers,4.1100,0.238747,4.587493,3.632507,18.689228,166.639482,7.168661,-147.950254,12.681122
2018-02-08,ZYLOG,3.85,4.0,3.80,3.9,107452,420336.55,175,Computers,4.0825,0.229000,4.540499,3.624501,16.413962,154.584705,6.514929,-138.170743,12.701292
2018-02-09,ZYLOG,3.75,4.0,3.65,3.9,119159,451916.80,205,Computers,4.0575,0.219614,4.496728,3.618272,14.488737,143.422875,5.991943,-128.934138,12.701292
2018-03-16,ZYLOG,3.65,3.8,3.65,3.7,182579,681707.50,131,Computers,4.0225,0.217930,4.458360,3.586640,12.828931,133.073033,5.533554,-120.244101,12.694794


In [93]:
# saving the dataframe to output csv file
df.to_csv('SMA_20_BB_GROUP.csv', index = True)